In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [2]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [ ]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [4]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, DuAv68EWrCrNof2giPx8bijG1YW5dQu4B3bHt2Xsfxmt, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [ ]:
# import pandas as pd
# 
# print("--")


In [5]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
   ["YYY", "Y-Airport"               , "Airport"],
   ["ZZZ", "Z-Airport"               , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

print(tabulate(df_airports, headers='keys', tablefmt='psql'))


+----+----------------+----------------+---------+
|    | airport_code   | airport_name   | LABEL   |
|----+----------------+----------------+---------|
|  0 | MKE            | Milwaukee      | Airport |
|  1 | ORD            | Chicago O-Hare | Airport |
|  2 | SJC            | San Jose       | Airport |
|  3 | DEN            | Denver         | Airport |
|  4 | YYY            | Y-Airport      | Airport |
|  5 | ZZZ            | Z-Airport      | Airport |
+----+----------------+----------------+---------+


In [6]:

# Import the previously created Python DataFrame into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiport set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   df_importer.insert()

print("--")


/opt/conda/lib/python3.8/site-packages/katana/remote/aio/import_data/dataframe_importer/__init__.py:235: UserWarning: unsupported dtype 'object' for 'airport_code' was coerced to 'string'
  warnings.warn(f"unsupported dtype '{original_dtype}' for '{id_col.name}' was coerced to '{id_col.dtype}'")


          0/? [?op/s]

--


In [8]:

l_query = """
   MATCH
      (n: Airport   )
   WHERE n.airport_code  < "Y"
   SET n.additional_property = 17
   """
      #
display(l_result = my_graph.query_unpaginated(l_query))


          0/? [?op/s]

In [22]:

#  What's in the graph ..

l_result1 = my_graph.query_unpaginated("""
   MATCH (n) 
   RETURN n
   """)
print(tabulate(l_result1, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+-------------+-----------+-------------------------+------------------+------------------+--------------+
|    |           id(n) | labels(n)   | n.LABEL   |   n.additional_property | n.airport_code   | n.airport_name   | n datatype   |
|----+-----------------+-------------+-----------+-------------------------+------------------+------------------+--------------|
|  0 |               0 | ['Airport'] | Airport   |                      17 | MKE              | Milwaukee        | node         |
|  1 |               2 | ['Airport'] | Airport   |                      17 | DEN              | Denver           | node         |
|  2 |               1 | ['Airport'] | Airport   |                      17 | SJC              | San Jose         | node         |
|  3 | 562949953421312 | ['Airport'] | Airport   |                      17 | ORD              | Chicago O-Hare   | node         |
+----+-----------------+-------------+-----------+-------------------------+--------------

In [21]:
#  Delete with count ..

l_result = my_graph.query_unpaginated("""

   MATCH (n: Airport)
   WHERE n.additional_property IS NULL
   WITH COUNT(n) AS num_deleted, COLLECT(n) AS nodes_deleted
   UNWIND nodes_deleted AS nodes
   DETACH DELETE nodes
   RETURN num_deleted AS num_deleted
   LIMIT 1
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

          0/? [?op/s]

+----+------------------+
|    | n.airport_code   |
|----+------------------|
|  0 | YYY              |
|  1 | ZZZ              |
+----+------------------+
